## Scraping boredapeyatchclub using selenium and beautiful soup & store to MongoDB

### Imports

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup 
import pymongo
import requests
import time
import json
import re

### Generic Functions

In [ ]:
def getdriver ():
    path=r'chromedriver.exe'
    #driver = webdriver.Chrome(executable_path='/Users/Sripriya Srinivasan/Downloads/chromedriver_win32/chromedriver')
    driver = webdriver.Chrome(executable_path=path)
    driver.implicitly_wait(10)
    driver.set_script_timeout(120)
    driver.set_page_load_timeout(30)
    return driver

In [ ]:
def writePage(fname,content):
    with open(fname, "w", encoding="utf-8") as file:
        file.write(str(content))

In [ ]:
def read_file (name):
    HTMLFile = open(name, "rb")
    htmlfiledata = HTMLFile.read()
    return BeautifulSoup(htmlfiledata, 'lxml')

In [ ]:
def connect_mongodb (db_name, collection_name):
    client = pymongo.MongoClient("mongodb://localhost:27017")
    db = client[db_name]
    collection = db[collection_name]
    return collection

In [ ]:
def loadWebsiteData (url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        page = requests.get(url,headers=headers)
        # Create a beautifulsoup object 
        return BeautifulSoup(page.text, 'lxml')
    except :
        print("Error connecting to website")

In [ ]:
def updateData (sr,new_val):
    collection = connect_mongodb ("pizzeria", "sf_pizzerias")
    myquery = { "search rank": sr }
    newvalues = { "$set": new_val }
    collection.update_many(myquery, newvalues)

## Selenium and MongoDB
### Task 1 
Using selenium access https://opensea.io/collection/boredapeyachtclubLinks with filter to select all apes with “Solid gold” fur and sort them “Price high to low” . Click on each of the top-8 most expensive Bored Apes, and store the resulting details page to disk, “bayc_[N].htm” (replace [N] with the ape number)

In [ ]:
ape_nums = []
def q2():
    url = "https://opensea.io/collection/boredapeyachtclub?search[sortAscending]=false&search[stringTraits][0][name]=Fur&search[stringTraits][0][values][0]=Solid%20Gold"
    driver = getdriver()
    driver.get(url)

    for i in range(0,8):
        time.sleep(2)
        ape = driver.find_elements("css selector","div.sc-29427738-0.sc-e7851b23-1.dVNeWL.hfa-DJE.Asset--loaded")[i]
        ape_num = driver.find_elements("css selector", "span.sc-29427738-0.sc-d58c749b-2.eNYnCu.heRZSz")[i].text
        ape_nums.append(ape_num)
        ape.click()
        time.sleep(5)
        content = driver.page_source
        fname = "bayc_"+str(ape_num)+".htm"
        time.sleep(2)
        writePage(fname,content)
        driver.execute_script("window.history.go(-1)")

    driver.close()

In [ ]:
q2()

### Task 2
Go through all 8 htm files downloaded in Task1 and store each ape’s name (its number) and all its attributes in a document inside a MongoDB collection called “bayc”.

In [ ]:
def q3():
    
    bayc = connect_mongodb('bayc','bayc')
    
    for ape_num in ape_nums:
        attr_list = []
        name = "bayc_"+str(ape_num)+".htm"
        soup = read_file(name)
        
        types = soup.select("div.Property--type")
        values = soup.select("div.Property--value")
        rarity = soup.select("div.Property--rarity")
        
        for i in range(0,len(types)):
            attr_list.append({types[i].text:{"value":values[i].text,"rarity":rarity[i].text}})
        
        ape_attr = {str(ape_num):attr_list}
        bayc.insert_one(ape_attr)
        print(ape_attr)
        print()

In [ ]:
q3()

In [ ]:
pizzeria_docs = q5()